### SocialAL Models
# Posterior Predictive Checks - double alpha model
KLS 7.15.22  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import beta
from scipy.stats import gamma
import matplotlib.pyplot as plt
import itertools

## Pull in functions

In [3]:
run common_functions.ipynb

In [4]:
run double_alpha_functions.ipynb

## Functions to simulate data

In [5]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [6]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [7]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn)
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    trial = []
    choices = []
    responses = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        
        # Trial
        trial.append(x+1)
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
    
        # after choice, update probability
        if choice != 1:
            Probs[t] = update_prob(response, Probs[t], a_gain, a_loss) 
        # then update value
        EVs[t] = update_value(Probs[t]) 

    data = {'Trial': trial, 'Stim_Sequence': trial_sequence, 'Choice' : choices, 'Trustee_Response': responses}    
    return data

In [8]:
#sim_data(15,[.2,.3,2])

### Pull in best-fit parameters

In [90]:
dt = pd.read_csv('../../output/two_alpha_model_params.csv')
dt.head(n=5)

,id,alpha_gain,alpha_loss,beta,-LLH
0,sub-2013,0.003498,0.002143,20.000000,46.155479
1,sub-2007,0.289955,0.206801,0.303044,51.566509
2,sub-2006,0.000418,0.000000,19.110026,60.782458
3,sub-2012,0.018243,1.000000,0.416436,47.261224
4,sub-2004,0.070699,0.131740,2.112863,32.966208


### Simulate data

In [91]:
data = pd.DataFrame(columns= ['Trial', 'Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject'])
for index, row in dt.iterrows():
    params = (row['alpha_gain'], row['alpha_loss'], row['beta'],)
    old_sub = row['id']
    for iter in range(100):
        dt = sim_data(15, params)
        dt['Subject'] = [old_sub + '-' + str(iter)] * 45
        dt = pd.DataFrame(dt)
        data = pd.concat([data, dt])
data

,Trial,Stim_Sequence,Choice,Trustee_Response,Subject
0,1,0,4,1,sub-2013-0
1,2,1,2,0,sub-2013-0
2,3,0,4,1,sub-2013-0
3,4,1,2,1,sub-2013-0
4,5,2,4,0,sub-2013-0
5,6,1,4,1,sub-2013-0
6,7,2,4,0,sub-2013-0
7,8,0,3,1,sub-2013-0
8,9,0,1,1,sub-2013-0
9,10,0,3,1,sub-2013-0


In [92]:
data.to_csv(path_or_buf = '../../output/ppc/sim_w_bestfit_2alpha_params.csv', index = False)